In [1]:
import sys
sys.path.append('../src')

import mlflow
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.model_selection import GridSearchCV # type: ignore


import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE
import pickle

from functions import *

from data_cleaning import DataCleaning

import os

pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('../data/raw_proform/2024_10_20.csv')

In [3]:
data.head(5)

,Race Time,Course,Distance (y),Race Type,Official Going,Classifications,Position,Horse,Jockey Claim,Weight (pounds),SP Odds Decimal,BF Decimal SP,"Won (1=Won, 0=Lost)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Decimal,PFR Rank,Main,Main Rank,Main Stats,Stats,Stats Rank,Trn Stats,Jky Stats,TrnJky Stats,Hrs Stats,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,4th LTOt Speed Rating,Evening Price,Breakfast Price,Morning Price,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Finished Races SR,WON SR Before,Won P/L Before,Plc SR Before,DSLR(F),DSLR(J),FLAT TURF Runs Before,FLAT AW Runs Before,CHASE Runs Before,HURDLE Runs Before,NHF Runs Before,ALL FLAT Runs Before,FLAT TURF Wins Before,FLAT AW Wins Before,CHASE Wins Before,HURDLE Wins Before,NHF Wins Before,Distance In Yards LTO,Official Rating LTO,Distance To Winner LTO,Betfair SP Odds LTO (Decimcal),Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,JSH Mean Movement Rank,Cherry Picked Rating,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Course Wins,Today's Course Places,Today's Course Runs,Heavy/Soft Ground Wins,Heavy/Soft Ground Places,Heavy/Soft Ground Runs,Good to Soft Ground Wins,Good to Soft Ground Runs,Good Ground Wins,Good Ground Runs,Good to Firm Ground Wins,Good to Firm Ground Runs,SpeedRating LTO
0,01/09/2024 2:00:00 PM,Tipperary,1980,Turf,Good,Mdn,1,Tennessee Stud,0,133,6.00,7.40,1,70,70,0,0,0,2,0,1,5.50,2,357,2,689,332,5,82,89,85,76,56,2,56,2,56,2,56,2,56,2,56,2,0,1,0,0,3.75,4.00,5.00,56.0,56.0,56.0,56.0,56.0,56.0,NaN,NaN,0.0,-1.0,0.0,29.0,NaN,1,0,0,0,0,1,0,0,0,0,0,1833,0.0,4.20,11.42,56.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,56.0
1,01/09/2024 2:00:00 PM,Tipperary,1980,Turf,Good,Mdn,2,Shackleton,0,133,1.44,1.55,0,87,87,0,0,2,1,0,1,2.38,1,373,1,734,361,2,97,84,88,92,63,1,63,1,63,1,63,1,63,1,63,1,0,1,0,0,1.57,1.53,1.44,63.0,63.0,63.0,63.0,63.0,63.0,NaN,NaN,0.0,-1.0,100.0,24.0,NaN,1,0,0,0,0,1,0,0,0,0,0,1760,0.0,5.00,5.20,63.0,0.0,0.0,0.0,0.0,NaN,NaN,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,63.0
2,01/09/2024 2:00:00 PM,Tipperary,1980,Turf,Good,Mdn,3,You Are Mine,0,128,81.00,170.19,0,50,50,0,0,0,2,0,1,7.50,5,331,4,640,309,6,74,83,76,76,46,3,46,3,46,3,46,3,46,3,46,3,0,1,0,0,34.00,34.00,51.00,46.0,46.0,46.0,46.0,46.0,46.0,NaN,NaN,0.0,-1.0,0.0,29.0,NaN,1,0,0,0,0,1,0,0,0,0,0,1833,0.0,14.45,44.00,46.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,46.0
3,01/09/2024 2:00:00 PM,Tipperary,1980,Turf,Good,Mdn,4,County Mayo,7,126,11.00,12.57,0,0,0,0,0,0,2,0,1,NaN,4,311,5,670,359,3,97,76,86,100,0,5,0,5,0,5,0,5,0,5,0,5,0,1,0,0,10.00,10.00,11.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
4,01/09/2024 2:00:00 PM,Tipperary,1980,Turf,Good,Mdn,5,Wolf Rayet,0,133,7.50,8.40,0,43,43,0,0,0,2,0,1,7.50,3,336,3,688,352,4,89,81,90,92,44,4,44,4,44,4,44,4,44,4,44,4,0,1,0,0,12.00,12.00,13.00,44.0,44.0,44.0,44.0,44.0,44.0,NaN,NaN,0.0,-1.0,0.0,48.0,NaN,1,0,0,0,0,1,0,0,0,0,0,1780,0.0,8.50,5.90,44.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,44.0


In [14]:
data['Race Time'].max()

'30/09/2024 5:35:00 PM'

In [4]:
def mass_transformation(data):
    bins = [800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600,
        2800, 3000, 3200, 3400, 3600, 3800, 4000]

    data['distance_bucket'] = pd.cut(data['Distance (y)'], bins = bins,
                               labels=['800-1000', '1000-1200', '1200-1400', '1400-1600', '1600-1800', '1800-2000',
                                       '2000-2200', '2200-2400', '2400-2600', '2600-2800', '2800-3000',
                                       '3000-3200', '3200-3400', '3400-3600', '3600-3800', '3800-4000'])
    
    data = data.rename(columns ={'Weight (pounds)':'weight'})

    data['evening_morning_price'] = (data['Morning Price'] / data['Evening Price'])

    data['breakfast_morning_price'] = (data['Morning Price'] / data['Breakfast Price'])

    data[['Won P/L Before', ]] = data[['Won P/L Before']].map(pd.to_numeric)

    data['SP Odds Decimal1'] = data['SP Odds Decimal']

    return data

def normalization(data):
    
    normalized_df = DataCleaning.normalize_columns(data, ['BF Decimal SP', 'LTO Speed Rating Rank', 
           'PFR Rank', 'Main Rank', 'HA Career Speed Rating Rank', "Today's Class Wins", "Today's Going Wins"])
    
    df1 = normalized_df[['BF Decimal SP', 'LTO Speed Rating Rank', 
           'PFR Rank', 'Main Rank', 'HA Career Speed Rating Rank', "Today's Class Wins", "Today's Going Wins"]]

    df1 = df1.fillna(0)

    return df1

In [5]:
df1 = mass_transformation(data)
df2 = normalization(df1)

In [6]:
df2.head()

,BF Decimal SP,LTO Speed Rating Rank,PFR Rank,Main Rank,HA Career Speed Rating Rank,Today's Class Wins,Today's Going Wins
0,0.006247,0.033333,0.033333,0.033333,0.033333,0.0,0.0
1,0.000390,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,0.169226,0.066667,0.133333,0.100000,0.066667,0.0,0.0
3,0.011423,0.133333,0.100000,0.133333,0.133333,0.0,0.0
4,0.007248,0.100000,0.066667,0.066667,0.100000,0.0,0.0


In [7]:
with open('../models/rf_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [8]:
y_probs = loaded_model.predict_proba(df2)[:, 1]
threshold = 0.53  # Example threshold
y_new_preds = (y_probs >= threshold).astype(int)

df1['model_preds'] = y_new_preds

profit_calculation(df1, stake = 1)

Total number of bets: 74
Total return from betting £1.00 on each prediction where model_preds == 1: £-58.29
Return per pound invested: £-0.79
Model accuracy: 5.41%


In [9]:
with open('../models/rf_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Use the loaded model to make predictions on new data
y_new_preds = loaded_model.predict(df2)

df1['model_preds'] = y_new_preds

profit_calculation(df1, stake = 1)

Total number of bets: 5953
Total return from betting £1.00 on each prediction where model_preds == 1: £-1673.30
Return per pound invested: £-0.28
Model accuracy: 10.38%


In [10]:
len(df1)

5958

In [11]:
df1.head()

,Race Time,Course,Distance (y),Race Type,Official Going,Classifications,Position,Horse,Jockey Claim,weight,SP Odds Decimal,BF Decimal SP,"Won (1=Won, 0=Lost)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Decimal,PFR Rank,Main,Main Rank,Main Stats,Stats,Stats Rank,Trn Stats,Jky Stats,TrnJky Stats,Hrs Stats,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,4th LTOt Speed Rating,Evening Price,Breakfast Price,Morning Price,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Finished Races SR,WON SR Before,Won P/L Before,Plc SR Before,DSLR(F),DSLR(J),FLAT TURF Runs Before,FLAT AW Runs Before,CHASE Runs Before,HURDLE Runs Before,NHF Runs Before,ALL FLAT Runs Before,FLAT TURF Wins Before,FLAT AW Wins Before,CHASE Wins Before,HURDLE Wins Before,NHF Wins Before,Distance In Yards LTO,Official Rating LTO,Distance To Winner LTO,Betfair SP Odds LTO (Decimcal),Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,JSH Mean Movement Rank,Cherry Picked Rating,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Course Wins,Today's Course Places,Today's Course Runs,Heavy/Soft Ground Wins,Heavy/Soft Ground Places,Heavy/Soft Ground Runs,Good to Soft Ground Wins,Good to Soft Ground Runs,Good Ground Wins,Good Ground Runs,Good to Firm Ground Wins,Good to Firm Ground Runs,SpeedRating LTO,distance_bucket,evening_morning_price,breakfast_morning_price,SP Odds Decimal1,model_preds
0,01/09/2024 2:00:00 PM,Tipperary,1980,Turf,Good,Mdn,1,Tennessee Stud,0,133,6.00,7.40,1,70,70,0,0,0,2,0,1,5.50,2,357,2,689,332,5,82,89,85,76,56,2,56,2,56,2,56,2,56,2,56,2,0,1,0,0,3.75,4.00,5.00,56.0,56.0,56.0,56.0,56.0,56.0,NaN,NaN,0.0,-1.0,0.0,29.0,NaN,1,0,0,0,0,1,0,0,0,0,0,1833,0.0,4.20,11.42,56.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,56.0,1800-2000,1.333333,1.250000,6.00,1
1,01/09/2024 2:00:00 PM,Tipperary,1980,Turf,Good,Mdn,2,Shackleton,0,133,1.44,1.55,0,87,87,0,0,2,1,0,1,2.38,1,373,1,734,361,2,97,84,88,92,63,1,63,1,63,1,63,1,63,1,63,1,0,1,0,0,1.57,1.53,1.44,63.0,63.0,63.0,63.0,63.0,63.0,NaN,NaN,0.0,-1.0,100.0,24.0,NaN,1,0,0,0,0,1,0,0,0,0,0,1760,0.0,5.00,5.20,63.0,0.0,0.0,0.0,0.0,NaN,NaN,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,63.0,1800-2000,0.917197,0.941176,1.44,1
2,01/09/2024 2:00:00 PM,Tipperary,1980,Turf,Good,Mdn,3,You Are Mine,0,128,81.00,170.19,0,50,50,0,0,0,2,0,1,7.50,5,331,4,640,309,6,74,83,76,76,46,3,46,3,46,3,46,3,46,3,46,3,0,1,0,0,34.00,34.00,51.00,46.0,46.0,46.0,46.0,46.0,46.0,NaN,NaN,0.0,-1.0,0.0,29.0,NaN,1,0,0,0,0,1,0,0,0,0,0,1833,0.0,14.45,44.00,46.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,46.0,1800-2000,1.500000,1.500000,81.00,1
3,01/09/2024 2:00:00 PM,Tipperary,1980,Turf,Good,Mdn,4,County Mayo,7,126,11.00,12.57,0,0,0,0,0,0,2,0,1,NaN,4,311,5,670,359,3,97,76,86,100,0,5,0,5,0,5,0,5,0,5,0,5,0,1,0,0,10.00,10.00,11.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1800-2000,1.100000,1.100000,11.00,1
4,01/09/2024 2:00:00 PM,Tipperary,1980,Turf,Good,Mdn,5,Wolf Rayet,0,133,7.50,8.40,0,43,43,0,0,0,2,0,1,7.50,3,336,3,688,352,4,89,81,90,92,44,4,44,4,44,4,44,4,44,4,44,4,0,1,0,0,12.00,12.00,13.00,44.0,44.0,44.0,44.0,44.0,44.0,NaN,NaN,0.0,-1.0,0.0,48.0,NaN,1,0,0,0,0,1,0,0,0,0,0,1780,0.0,8.50,5.90,44.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,44.0,1800-2000,1.083333,1.083333,7.50,1


## next set

In [12]:
df2 = mass_transformation(data1)
df3 = normalization(df2)

NameError: name 'data1' is not defined

In [15]:
# Use the loaded model to make predictions on new data
y_new_preds = loaded_model.predict(df3)

df2['model_preds'] = y_new_preds

profit_calculation(df2, stake = 1)

Total number of bets: 171
Total return from betting £1.00 on each prediction where model_preds == 1: £120.66
Return per pound invested: £0.71
Model accuracy: 38.60%


## next set

In [17]:
df22 = mass_transformation(data2)
df33 = normalization(df22)

In [23]:


# Use the loaded model to make predictions on new data
y_new_preds = loaded_model.predict(df33)

df22['model_preds'] = y_new_preds

profit_calculation(df22, stake = 2)

Total number of bets: 1094
Total return from betting £2.00 on each prediction where model_preds == 1: £1919.48
Return per pound invested: £0.88
Model accuracy: 32.91%
